# NPCC Cost

In [1]:
from tqdm import tqdm

import numpy as np
import pandas as pd

import andes
import ams

import datetime

import matplotlib
import matplotlib.pyplot as plt
import scienceplots

In [2]:
matplotlib.rcdefaults()

In [3]:
%matplotlib inline

In [4]:
print("Last run time:", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

print(f'andes:{andes.__version__}')
print(f'ams:{ams.__version__}')

Last run time: 2024-04-12 21:13:14
andes:1.9.1.post37+g35b2a678
ams:0.9.5.post25+g738d558


In [5]:
andes.config_logger(stream_level=20)
ams.config_logger(stream_level=20)

In [6]:
ams_file = './data/npcc_uced_base.xlsx'

sp = ams.load(ams_file,
              setup=True, no_output=True, default_config=True,)
sp0 = ams.load(ams_file,
               setup=True, no_output=True, default_config=True,)

idx_pq = sp.PQ.idx.v
p0_pq = sp.PQ.p0.v.copy()
q0_pq = sp.PQ.q0.v.copy()

Working directory: "/Users/jinningwang/Documents/work/psal/src/notes/lmp"
Parsing input file "./data/npcc_uced_base.xlsx"...
Input file parsed in 0.2174 seconds.
Zero line rates detacted in rate_b, rate_c, adjusted to 999.
System set up in 0.0030 seconds.
Working directory: "/Users/jinningwang/Documents/work/psal/src/notes/lmp"
Parsing input file "./data/npcc_uced_base.xlsx"...
Input file parsed in 0.1721 seconds.
Zero line rates detacted in rate_b, rate_c, adjusted to 999.
System set up in 0.0028 seconds.


In [7]:
gen_idx = sp.RTED.pg.get_idx()
cols = ['97IBR', '97IBRI', '105IBR', '105IBRI']

pgc = pd.DataFrame(data=-1*np.ones((len(gen_idx), len(cols))),
                   columns=cols, index=gen_idx)

In [8]:
# alter the load level for both systems
# 0.0.97, 1.05
k_load = 1.05 # load factor
sp.PQ.alter(src='p0', idx=idx_pq, value=p0_pq * k_load)
sp.PQ.alter(src='q0', idx=idx_pq, value=q0_pq * k_load)
sp0.PQ.alter(src='p0', idx=idx_pq, value=p0_pq * k_load)
sp0.PQ.alter(src='q0', idx=idx_pq, value=q0_pq * k_load)

# hard coded
gen_vsg = ['PV_21', 'PV_23', 'PV_25', 'PV_26']
gcost_vsg = sp.GCost.find_idx(keys='gen', values=gen_vsg)
n_vsg = len(gen_vsg)

# alter the cost for only IBR
sp.GCost.set(src='c2', attr='v', idx=gcost_vsg, value=np.zeros(n_vsg))
sp.GCost.set(src='c1', attr='v', idx=gcost_vsg, value=np.zeros(n_vsg))
sp.GCost.set(src='c0', attr='v', idx=gcost_vsg, value=np.zeros(n_vsg))

sp.RTED.init()
sp.RTED.run(solver='MOSEK')

sp0.RTED.init()
sp0.RTED.run(solver='MOSEK')

# pgc['97IBR'] = sp.RTED.get(src='pg', attr='v', idx=gen_idx)
# pgc['97IBRI'] = sp0.RTED.get(src='pg', attr='v', idx=gen_idx)

pgc['105IBR'] = sp.RTED.get(src='pg', attr='v', idx=gen_idx)
pgc['105IBRI'] = sp0.RTED.get(src='pg', attr='v', idx=gen_idx)

<RTED> initialized in 0.0249 seconds.
<RTED> solved as optimal in 0.0420 seconds, converged in 13 iterations with MOSEK.
<RTED> initialized in 0.0173 seconds.
<RTED> solved as optimal in 0.0387 seconds, converged in 16 iterations with MOSEK.


In [9]:
# save to csv file
# pgc.to_csv('./res/pg.csv')